# Time To Move Data with Spark

#### First Let'st create connection to our SQL Db & our Blob Storage

In [0]:
print('Process Started')

# Define the JDBC connection properties using your provided connection string
jdbc_url = "jdbc:sqlserver://databaseserver.database.windows.net:1433;database=yourdatabasename"
connection_properties = {
    "user": "databaseusername",  # Your username
    "password": "password",  # Replace with your actual password
    "driver": "com.microsoft.sqlserver.jdbc.SQLServerDriver",
    "encrypt": "true",
    "trustServerCertificate": "false",
    "hostNameInCertificate": "*.database.windows.net",
    "loginTimeout": "30"
}
print("Connection to db succesfully")

# Define your Azure Storage account name and key
storage_account_name = "storagename"
storage_account_key = "youraccountkey"
container_name = "containername"  

# Define your mount point
mount_point = f"/mnt/{container_name}"

# Check if the storage is already mounted
if any(mount.mountPoint == mount_point for mount in dbutils.fs.mounts()):
    print(f"{mount_point} is already mounted.")
else:
    dbutils.fs.mount(
      source = f"wasbs://{container_name}@{storage_account_name}.blob.core.windows.net",
      mount_point = mount_point,
      extra_configs = {f"fs.azure.account.key.{storage_account_name}.blob.core.windows.net": storage_account_key}
    )
    print(f"Mounted {mount_point} successfully.")

Process Started
Connection to db succesfully
/mnt/americanagg-container is already mounted.


#### Function Time
With this Function aim to read the our file from blob Storage

In [0]:
def read_csv(csv_file_path):
    df = spark.read.format("csv").option("header", "true").load(csv_file_path)
    return df

# File path to the CSV file
try:
    file_path = "/mnt/americanagg-container/Business Conditions Major.csv"
    z = read_csv(file_path)
    print('File successfully read')
except Exception as e:
    print(f'Error reading file: {e}')

File successfully read


# Spark Magic
With a simple lines of code spark will bulk the information into our SQL db

### NOTE:
The number of columns and names should match the table in sql server,spark itself will figure it out how to insert the data.

In [0]:
table_name = "[yourtablename]"
z.write.jdbc(url=jdbc_url, table=table_name, mode="append", properties=connection_properties)
print("data succesfully Inserted")

data succesfully Inserted
